In [1]:
library(readxl)
library(data.table)
library(ggplot2)
library(ggprism)
library(ggforce)
library(ggetho)
library(ggsignif)
library(zoo)
library(tidyxl)
library(magrittr)
theme_set(theme_prism(base_size=20))
options(repr.plot.width=15, repr.plot.height=15)

Loading required package: behavr


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric




In [2]:
idoc_dir <- "/idoc_data/results/IDOC_001/"
xlsx_file <- "/idoc_data/poster/IDOC Experiment.xlsx"

In [3]:
chui_sheets <- readxl::excel_sheets(xlsx_file)

In [4]:
chui_sheets <- chui_sheets[2:length(chui_sheets)]

In [1113]:
# chui_sheets <- c("ISO31 CHX Odor", "Iso31 Odor LTM", "KC GFP odor learning", "KC Odor learning", "KC odor learning unpaired", "KC Odor LTM", "orb2 mutant odor LTM", "Orb2 odor learning", "SPARC KC odor CHX")

In [1114]:
# chui_sheets <- "Arc1 esm18"

In [1115]:
# for (sheet in chui_sheets) {
#     if (length(grep(pattern = "Odor|odor|Odour|odour", x = sheet)) > 0) {
#         print(paste0("Odor -> ", sheet))
#     } else if (length(grep(pattern = "LED|led", x = sheet)) > 0) {
#         print(paste0("LED -> ", sheet))
#     } else {
#         print(paste0("Unknown -> ", sheet))
#     }
# }

In [1116]:
columns <- c("Files", "Genotype", "ROI", "Gender", "Age", "ATR", "Training", "Light (627nm)", "Drug", "Odor", "Sugar", "Electric shock", "Fasting", "sheet", "row_number", "PRE", "POST")

In [1117]:
rm(chui_metadata)

In [1118]:
chui_metadata <- lapply(
    chui_sheets,
    function(sheet) {
        dt <- tryCatch({
            dt <- suppressMessages(readxl::read_xlsx(xlsx_file, sheet = sheet))
            
            first_cell <- unname(unlist(dt[1,1]))

            if(!is.na(first_cell) & first_cell  == "Files") {
                colnames(dt) <- unname(unlist(dt[1,]))                
                dt <- dt[2:nrow(dt), ]
            }
            dt$sheet <- sheet
            dt$Files <- as.character(dt$Files)
            dt$row_number <- 1:nrow(dt)
            dt[which(dt$Files == "Learning"),]
            dt[which(dt$Files == "Learning"), "Files"] <- NA
            dt <- as.data.table(dt)
            
            stopifnot(dt[ROI==1, sum(is.na(Files)) == 0])
            stopifnot(all(rle(dt$Files)$length < 20))
            
            dt[, Files := na.locf(Files)]
   
            
            if (all(is.na(dt$Genotype))) {
                dt[, Genotype := sheet]
            }
            dt[, Genotype := na.locf(Genotype)]
            
            selected_cols <- which(sapply(colnames(dt), function(x) {substr(x, 1, 7) == "Counted"}))
            dt <- dt[(
                sapply(dt[[names(selected_cols[1])]], function(x) x %in% c("V", "OK")) &
                sapply(dt[[names(selected_cols[2])]], function(x) x %in% c("V", "OK"))
                    ), ]

            pi_cols <- sort(which(sapply(colnames(dt), function(x) {substr(x, 1, 7) == "Average"})))
            stopifnot(length(pi_cols)==2)
            colnames(dt)[pi_cols[1]] <- "PRE"
            colnames(dt)[pi_cols[2]] <- "POST"
            dt$POST <- as.numeric(dt$POST)
            dt$PRE <- as.numeric(dt$PRE)
            dt    
            
            
        }, error = function(e) {
            print(e)
            print(sheet)
            NULL
        })
        dt
        
    }
)


Warning message in doTryCatch(return(expr), name, parentenv, handler):
“NAs introduced by coercion”
Warning message in doTryCatch(return(expr), name, parentenv, handler):
“NAs introduced by coercion”
Warning message:
“Expecting numeric in A2 / R2C1: got a date”
Warning message:
“Expecting numeric in A22 / R22C1: got a date”


<simpleError in FUN(X[[i]], ...): dt[ROI == 1, sum(is.na(Files)) == 0] is not TRUE>
[1] "R45H04 SPACED STM COLD"


In [1119]:
names(chui_metadata) <- chui_sheets

In [1120]:
out0 <- Reduce(x = chui_metadata, f = function(x, y) {
    if (is.null(y)) {
        x
    } else if (nrow(y) == 0) {
        x
    } else if ("data.frame" %in% class(y)) {
        new_cols_y <- setdiff(colnames(y), colnames(x))
        new_cols_x <- setdiff(colnames(x), colnames(y))
        new_header <- unique(c(colnames(x), colnames(y)))

        for (col in new_cols_x) {
             y[[col]] <- NA
        }
        for (col in new_cols_y) {
             x[[col]] <- NA
        }
        rbind(x, y)
    } else {
        x
    }
})

In [1121]:
out <- as.data.table(out0[, columns[columns%in%colnames(out0)], with=F])

In [1122]:
out[Files == "44597", "Files"] <- as.character(as.Date(44597, origin = "1899-12-30"))
out[Files == "44594", "Files"] <- as.character(as.Date(44594, origin = "1899-12-30"))
out[Files == "44581", "Files"] <- as.character(as.Date(44581, origin = "1899-12-30"))
out[Files == "44565", "Files"] <- as.character(as.Date(44565, origin = "1899-12-30"))
out[Files == "44564", "Files"] <- as.character(as.Date(44564, origin = "1899-12-30"))
out[Files == "44547", "Files"] <- as.character(as.Date(44547, origin = "1899-12-30"))
out[Files == "44538", "Files"] <- as.character(as.Date(44538, origin = "1899-12-30"))
out[Files == "44535", "Files"] <- as.character(as.Date(44535, origin = "1899-12-30"))
out[Files == "44533", "Files"] <- as.character(as.Date(44533, origin = "1899-12-30"))
out[Files == "44200", "Files"] <- as.character(as.Date(44200, origin = "1899-12-30"))
out[Files == "44527", "Files"] <- as.character(as.Date(44527, origin = "1899-12-30"))
out[Files == "44528", "Files"] <- as.character(as.Date(44528, origin = "1899-12-30"))
out[Files == "44532", "Files"] <- as.character(as.Date(44532, origin = "1899-12-30"))
out[Files == "44553", "Files"] <- as.character(as.Date(44553, origin = "1899-12-30"))
out[Files == "44558", "Files"] <- as.character(as.Date(44558, origin = "1899-12-30"))
out[Files == "44503", "Files"] <- as.character(as.Date(44503, origin = "1899-12-30"))
out[Files == "44508", "Files"] <- as.character(as.Date(44508, origin = "1899-12-30"))

out[Files == "44474", "Files"] <- as.character(as.Date(44474, origin = "1899-12-30"))
out[Files == "44475", "Files"] <- as.character(as.Date(44475, origin = "1899-12-30"))
out[Files == "44476", "Files"] <- as.character(as.Date(44476, origin = "1899-12-30"))
out[Files == "44504", "Files"] <- as.character(as.Date(44504, origin = "1899-12-30"))
out[Files == "44498", "Files"] <- as.character(as.Date(44498, origin = "1899-12-30"))
out[Files == "44501", "Files"] <- as.character(as.Date(44501, origin = "1899-12-30"))

out[Files == "44457", "Files"] <- as.character(as.Date(44457, origin = "1899-12-30"))
out[Files == "44552", "Files"] <- as.character(as.Date(44552, origin = "1899-12-30"))

out[Files == "44458", "Files"] <- as.character(as.Date(44458, origin = "1899-12-30"))
out[Files == "44470", "Files"] <- as.character(as.Date(44470, origin = "1899-12-30"))
out[Files == "44473", "Files"] <- as.character(as.Date(44473, origin = "1899-12-30"))
44473

out[Files == "20220122", "Files"] <- "2022-01-22"
out[Files == "2022/0513", "Files"] <- "2022-05-13"
out[Files == "2022/0416", "Files"] <- "2022-04-16"


[1] 44473

In [1123]:
metadata <-data.table::copy(out)

In [1124]:
metadata[sheet == "ISO31 CHX Odor", unique(Files)]

[1] "2021-11-27 (CH1)" "2021-11-28"       "2021-11-30 (CH)"  "2021-12-02"      
[5] "2021-12-17 (CH)"  "2021-12-23"       "2021-12-28 (CH)"

In [1125]:
metadata <- metadata[-grep(x = metadata$sheet, pattern="LED|led"),]
metadata <- metadata[!(sheet == "Sheet1"),]
metadata <- metadata[!(sheet == "KC Learning"),]
metadata <- metadata[is.na(`Light (627nm)`) | `Light (627nm)` == "N", ]
metadata[, `Light (627nm)` := NULL]
# metadata <- metadata[is.na(ATR) | ATR == "N", ]
# metadata[, ATR := NULL]
metadata <- metadata[is.na(Sugar) | Sugar == "N", ]
metadata[, Sugar := NULL]
metadata <- metadata[is.na(Fasting) | Fasting == "N", ]
metadata[, Fasting := NULL]
metadata[, `Electric shock` := NULL]
metadata[Training=="Spaced x 6", Training := "6X"]


metadata<-metadata[Genotype != "010B CREB LTM Heat",]
metadata<-metadata[Genotype != "010B CREB-2b LTM RT",]
metadata<-metadata[Genotype!="Radish",]
colnames(metadata)[colnames(metadata) == "Training"] <- "Training_session"
metadata[grep(pattern = "Orb|orb", x = Genotype), Genotype := "orb2"]

In [1126]:
metadata<-metadata[!(Files=="2021-12-17" & Genotype == "orb2"),]

In [1127]:
link_idoc_metadata <- function(metadata, result_dir) {

    metadata <- data.table::copy(metadata)
    metadata <- metadata[order(sheet, Files),]
    
    metadata[, folder__ := Files]
    metadata[, sheet__ := sheet]
    
    metadata_unique <- metadata[, .SD[1], by=.(Files, sheet)]
    
    metadata_unique$idoc_folder <- NA_character_
    metadata_unique <- metadata_unique[, link_idoc_file(.SD, result_dir=result_dir), by=.(Files, sheet)]

    
    metadata_linked <- merge(metadata, metadata_unique[, .(Files, sheet, idoc_folder)], by=c("Files", "sheet"))

    return(metadata_linked)
}

link_idoc_file <- function(metadata, result_dir) {
    # print(paste0(metadata[, .(sheet__, folder__)], collapse=" "))
    
    year <- substr(metadata$folder__, 1, 4)
    if (!(year %in% c("2021", "2022"))) {
        return(NULL)
    }
    if (year != "2021") {
        print(metadata)
    }
    
    month <- substr(metadata$folder__, 6, 7)
    month_folder <- paste0(year, "-", month, "-idoc_data")
    


    x <- file.path(result_dir, month_folder)
    month_folders <- list.files(x, full.names = TRUE)
    hits <- c()
    for (folder in month_folders) {
        if (metadata$folder__ == basename(folder)) {
            hits <- c(folder)
            break
        }
        
        if (length(grep(pattern = metadata$folder__, x = folder)) > 0) {
            hits <- c(hits, folder)
        }
        
    }
    
    out <- data.table::copy(metadata)
    
    if (length(hits) == 1) {
        message(paste0("1 hit found for ", metadata$sheet_, "/", metadata$folder__))
        out[, idoc_folder := hits[1]]
    } else if (length(hits) == 0) {
        message(paste0("0 hits found for ", metadata$sheet_, "/",  metadata$folder__))
        out[, idoc_folder := NA_character_]
    } else {
        message(paste0(length(hits), " hits found for ", metadata$sheet_, "/",  metadata$folder__))
        print(metadata[, .(sheet__, folder__)])
        print(paste0(metadata$folder__, ": ", paste0(hits, collapse=", ")))
        out[, idoc_folder := NA_character_]        
    }
    return(out)
}

In [1128]:
metadata[sheet=="Orb2 odor learning", table(Files)]

Files
2021-11-24 2022-06-23 
         7         14 

In [1129]:
metadata_linked <- link_idoc_metadata(metadata, result_dir="/idoc_data/IDOC")

1 hit found for ISO31 CHX Odor/2021-11-27 (CH1)

1 hit found for ISO31 CHX Odor/2021-11-28

1 hit found for ISO31 CHX Odor/2021-11-30 (CH)

1 hit found for ISO31 CHX Odor/2021-12-02

1 hit found for ISO31 CHX Odor/2021-12-17 (CH)

0 hits found for ISO31 CHX Odor/2021-12-23

1 hit found for ISO31 CHX Odor/2021-12-28 (CH)

1 hit found for Iso31 Odor LTM/2021-11-03 (CH1)

1 hit found for Iso31 Odor LTM/2021-11-08

1 hit found for Iso31 Odor LTM/2021-11-17 (CH)

1 hit found for Iso31 Odor LTM/2021-11-17 (CH1)

1 hit found for KC GFP odor learning/2021-10-02 (CH)

1 hit found for KC GFP odor learning/2021-10-02 (CH1)

1 hit found for KC GFP odor learning/2021-10-06 (CH)

1 hit found for KC Odor LTM/2021-10-05

1 hit found for KC Odor LTM/2021-10-06 (CH)

1 hit found for KC Odor LTM/2021-10-06 (CH1)

1 hit found for KC Odor LTM/2021-10-07

1 hit found for KC Odor LTM/2021-10-27 (CH1)

1 hit found for KC Odor LTM/2021-11-04

1 hit found for KC Odor learning/2021-09-30 (CH)

1 hit found for KC

   Genotype ROI Gender Age ATR Training_session Drug Odor row_number   PRE POST
1:     orb2   1      F   4   N               1X <NA>  OCT         11 0.125    0
     folder__            sheet__ idoc_folder
1: 2022-06-23 Orb2 odor learning        <NA>


3 hits found for Orb2 odor learning/2022-06-23



              sheet__   folder__
1: Orb2 odor learning 2022-06-23
[1] "2022-06-23: /idoc_data/IDOC/2022-06-idoc_data/2022-06-23_00-00-00(CH), /idoc_data/IDOC/2022-06-idoc_data/2022-06-23_11-59-19(ESB1), /idoc_data/IDOC/2022-06-idoc_data/2022-06-23_13-57-14(ESB2)"


1 hit found for SPARC KC odor CHX/2021-10-27 (CH)

1 hit found for SPARC KC odor CHX/2021-10-29

1 hit found for SPARC KC odor CHX/2021-11-01 (CH)

1 hit found for SPARC KC odor CHX/2021-11-01 (CH1)

1 hit found for orb2 mutant odor LTM/2021-12-03 (CH)

1 hit found for orb2 mutant odor LTM/2021-12-05 (CH)

1 hit found for orb2 mutant odor LTM/2021-12-08 (CH)

1 hit found for orb2 mutant odor LTM/2021-12-22 (CH1)



   Genotype ROI Gender Age ATR Training_session Drug Odor row_number PRE POST
1:     orb2   4      F   4   N               6X    N  OCT         61   0 -0.3
     folder__              sheet__ idoc_folder
1: 2022-01-03 orb2 mutant odor LTM        <NA>


3 hits found for orb2 mutant odor LTM/2022-01-03



                sheet__   folder__
1: orb2 mutant odor LTM 2022-01-03
[1] "2022-01-03: /idoc_data/IDOC/2022-01-idoc_data/2022-01-03 (CH), /idoc_data/IDOC/2022-01-idoc_data/2022-01-03 (CH1), /idoc_data/IDOC/2022-01-idoc_data/2022-01-03 (CH2)"


In [1130]:
# the folder is missing but we have the PIs in CH spreadsheet
# I assign an idoc_folder here manually to prevent it from being removed
# All other flies without a matching IDOC folder will be removed
metadata_linked[Files == "2021-12-23", idoc_folder := "2021-12-23"]
metadata_linked[Files == "2021-11-28", idoc_folder := "2021-11-28"]


In [1131]:
mean(is.na(metadata_linked$idoc_folder))

[1] 0.06583072

In [1132]:
# metadata_linked <- metadata_linked[!is.na(idoc_folder),]

In [1133]:
metadata_linked[, table(sheet)]

sheet
           ISO31 CHX Odor            Iso31 Odor LTM      KC GFP odor learning 
                       38                        31                        38 
         KC Odor learning KC odor learning unpaired               KC Odor LTM 
                       60                        54                        31 
     orb2 mutant odor LTM        Orb2 odor learning         SPARC KC odor CHX 
                       21                        21                        25 

In [1134]:
metadata_linked[, experiment := NA_character_]

In [1135]:
metadata_linked[grep(x = sheet, pattern="LTM"), experiment := "LTM"]

In [1136]:
metadata_linked[grep(x = sheet, pattern="CHX"), experiment := "LTM CHX"]

In [1137]:
metadata_linked[grep(x = sheet, pattern="learning"), experiment := "20min STM"]
metadata_linked$Pairing <- "Paired"
metadata_linked[grep(x = sheet, pattern="learning unpaired"), experiment := "20min STM unpaired"]
metadata_linked[grep(x = sheet, pattern="learning unpaired"), Pairing := "unpaired"]

In [1138]:
table(metadata_linked$experiment)


         20min STM 20min STM unpaired                LTM            LTM CHX 
               119                 54                 83                 63 

In [1139]:
mean(is.na(metadata_linked$experiment))

[1] 0

In [1140]:
table(metadata_linked$Genotype)


                        Iso31 MB010B.(II)SPARC-Chrimson ISO 
                           69                           170 
     MB010B.(II)SPARC-GFP ISO                          orb2 
                           38                            42 

In [1141]:
mean(is.na(metadata_linked$Genotype))

[1] 0

In [1142]:
metadata_linked[, fly_name_reference := paste0(basename(idoc_folder), "_ROI_", ROI)]

In [1143]:
metadata_linked[, Fly_group_experiment_treatment := paste0(Genotype, experiment, "_", Odor, "_Csplus_EShock_US_", Pairing, "_", Training_session, "_ETHOSCOPE_NO")]

In [1144]:
colnames(metadata_linked)

[1] "Files"                          "sheet"                         
 [3] "Genotype"                       "ROI"                           
 [5] "Gender"                         "Age"                           
 [7] "ATR"                            "Training_session"              
 [9] "Drug"                           "Odor"                          
[11] "row_number"                     "PRE"                           
[13] "POST"                           "folder__"                      
[15] "sheet__"                        "idoc_folder"                   
[17] "experiment"                     "Pairing"                       
[19] "fly_name_reference"             "Fly_group_experiment_treatment"

In [1145]:
read_pi <- function(path, roi) {
   pis <- data.table::fread(path)[aversive + apetitive >= 3,]
   pi <- pis[region_id == roi, preference_index]
   if (length(pi) == 0) {
       pi <- NA
   }
   return(pi)
    
}

load_chui_data <- function(metadata, ncores=1) {
    data <- do.call(rbind, parallel::mclapply(1:nrow(metadata), function(i) {
       meta <- metadata[i, ]
       region_id <- meta$ROI
       subexperiments <- sort(list.files(meta$idoc_folder, full.names = TRUE))
       pre_data <- subexperiments[1]
       post_data <- subexperiments[length(subexperiments)]
       pre_pi_file <- list.files(pre_data, pattern = "_PI.csv", full.names = TRUE)
       post_pi_file <- list.files(post_data, pattern = "_PI.csv", full.names = TRUE)
       
       if (length(pre_pi_file) != 1) {
           warning(paste0("Cannot find PRE for ", meta$idoc_folder, "_ROI_", region_id))
           return(NULL)
       }
       if (length(post_pi_file) != 1) {
           warning(paste0("Cannot find POST for ", meta$idoc_folder, "_ROI_", region_id))
           return(NULL)
       }
                
       meta$PRE <- read_pi(pre_pi_file, region_id)
       meta$POST <- read_pi(post_pi_file, region_id)
       meta
    }, mc.cores=ncores))
    return(data)    
}

In [1146]:
metadata_linked$Training_session <- gsub(pattern = "6X", replacement = "6X_Spaced", x = metadata_linked$Training_session)

In [1147]:
table(metadata_linked$sheet)


           ISO31 CHX Odor            Iso31 Odor LTM      KC GFP odor learning 
                       38                        31                        38 
         KC Odor learning KC odor learning unpaired               KC Odor LTM 
                       60                        54                        31 
     orb2 mutant odor LTM        Orb2 odor learning         SPARC KC odor CHX 
                       21                        21                        25 

In [1148]:
# metadata_linked[Genotype=="orb2" & experiment == "LTM",]

In [1149]:
metadata_linked[Genotype=="Iso31" & experiment == "LTM", table(sheet)]

sheet
Iso31 Odor LTM 
            31 

In [1150]:
metadata_linked[Genotype=="Iso31" & experiment == "LTM CHX", table(sheet)]

sheet
ISO31 CHX Odor 
            38 

In [1151]:
# use raw csv
# data <- load_chui_data(metadata_linked)
# use chui annotation in excel file
data <- data.table::copy(metadata_linked)

In [1152]:
data <- data[!is.na(PRE) & !is.na(POST),] 

In [1153]:
nrow(data)

[1] 318

In [1154]:
#data <- data[abs(PRE) < 0.2,]

In [1155]:
nrow(data)

[1] 318

In [1156]:
zts <- paste0("ZT", seq(4, 30, 0.5))
for (zt in zts) {
    data[[zt]] <- NA
}
for (zt in zts) {
    data[[paste0(zt, "_interactions")]] <- NA
}

In [1157]:
data$id <- 1:nrow(data)

In [1158]:
data$interval <- "NONE"

In [1159]:
data <- melt(data, value.name = "PI", measure.vars = c("PRE", "POST"), variable.name = "test")

In [1160]:
data[abs(PI)>1,]

Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”


Files,sheet,Genotype,ROI,Gender,Age,ATR,Training_session,Drug,Odor,⋯,ZT27.5_interactions,ZT28_interactions,ZT28.5_interactions,ZT29_interactions,ZT29.5_interactions,ZT30_interactions,id,interval,test,PI
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,⋯,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<int>,<chr>,<fct>,<dbl>


In [1161]:
# panel1_data <- data[interval=="NONE" & experiment %in% c("20min STM") & Genotype %in% c("Iso31", "MB010B.(II)SPARC-Chrimson ISO", "MB010B.(II)SPARC-GFP ISO"),]
# panel1_data <- panel1_data[Genotype == 'MB010B.(II)SPARC-Chrimson ISO',]
# dcast(panel1_data[, .(fly_name_reference = substr(fly_name_reference, 1, 15), ROI, PI, test)], formula = fly_name_reference+ ROI ~ test, value.var = "PI")
# source("library.R", local = T)
# options(repr.plot.width=3)
# annotation_df <- make_annotation_df(panel1_data, "experiment", alternative="greater")
# panel1 <- ggplot(data=panel1_data, aes(x=test, y = PI, group=experiment)) +
#     geom_point(size=5) +
#     geom_line(aes(group=id)) + geom_pop_etho() +
#     scale_y_continuous(breaks=seq(-1, 1, 0.5), limits=c(-1, 1)) +
#     scale_x_discrete(expand = expansion(mult = c(0.1, 0.1))) +
#     facet_grid(. ~ experiment)

# panel1 <- panel1 + geom_text(data=annotation_df, y=-0.8, size=5, x="PRE", hjust=-.5, mapping=aes(label = paste0("N = ", N/2))) +
#     geom_signif(
#         data = annotation_df, 
#         aes(annotations = annotations), xmin="PRE", xmax="POST",
#         y_position = 0.8,
#         manual = TRUE,  tip_length = 0
#     )
# panel1

In [1162]:
data[, table(experiment)]
data[experiment == "20min STM", table(sheet)]

experiment
         20min STM 20min STM unpaired                LTM            LTM CHX 
               238                108                164                126 

sheet
KC GFP odor learning     KC Odor learning   Orb2 odor learning 
                  76                  120                   42 

In [1163]:
data[Genotype=="orb2", table(experiment)]

experiment
20min STM       LTM 
       42        42 

In [1164]:
data <- data[, c("Fly_group_experiment_treatment", "fly_name_reference",	"experiment", "id", "test", "PI", "Training_session", "Genotype", "interval", zts, paste0(zts, "_interactions")), with=F]

In [1165]:
data[Genotype=="Iso31" & experiment == "LTM" & test == "PRE", fly_name_reference]

[1] "2021-11-03 (CH1)_ROI_1"  "2021-11-03 (CH1)_ROI_2" 
 [3] "2021-11-03 (CH1)_ROI_3"  "2021-11-03 (CH1)_ROI_5" 
 [5] "2021-11-03 (CH1)_ROI_6"  "2021-11-03 (CH1)_ROI_7" 
 [7] "2021-11-03 (CH1)_ROI_8"  "2021-11-03 (CH1)_ROI_9" 
 [9] "2021-11-08 (CH)_ROI_1"   "2021-11-08 (CH)_ROI_2"  
[11] "2021-11-08 (CH)_ROI_3"   "2021-11-08 (CH)_ROI_5"  
[13] "2021-11-08 (CH)_ROI_6"   "2021-11-08 (CH)_ROI_7"  
[15] "2021-11-08 (CH)_ROI_8"   "2021-11-08 (CH)_ROI_9"  
[17] "2021-11-08 (CH)_ROI_10"  "2021-11-17 (CH)_ROI_1"  
[19] "2021-11-17 (CH)_ROI_2"   "2021-11-17 (CH)_ROI_3"  
[21] "2021-11-17 (CH)_ROI_4"   "2021-11-17 (CH)_ROI_5"  
[23] "2021-11-17 (CH)_ROI_7"   "2021-11-17 (CH)_ROI_8"  
[25] "2021-11-17 (CH)_ROI_9"   "2021-11-17 (CH1)_ROI_3" 
[27] "2021-11-17 (CH1)_ROI_4"  "2021-11-17 (CH1)_ROI_5" 
[29] "2021-11-17 (CH1)_ROI_7"  "2021-11-17 (CH1)_ROI_8" 
[31] "2021-11-17 (CH1)_ROI_10"

In [1166]:
data.table::fwrite(x = data, file = "chui_tidy_data.csv")

In [1167]:
table(data$experiment)


         20min STM 20min STM unpaired                LTM            LTM CHX 
               238                108                164                126 

In [1168]:
data[experiment == "20min STM",table(Genotype)]

Genotype
MB010B.(II)SPARC-Chrimson ISO      MB010B.(II)SPARC-GFP ISO 
                          120                            76 
                         orb2 
                           42 

In [1169]:
data[experiment == "20min STM" & test == "PRE", table(Genotype)]

Genotype
MB010B.(II)SPARC-Chrimson ISO      MB010B.(II)SPARC-GFP ISO 
                           60                            38 
                         orb2 
                           21 

In [1170]:
data[Genotype=="orb2", table(experiment)]

experiment
20min STM       LTM 
       42        42 